In [85]:
import numpy as np 
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder, MinMaxScaler , OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import FeatureUnion, Pipeline 
from sklearn.preprocessing import PowerTransformer
from sklearn.experimental import enable_iterative_imputer

from sklearn.impute import IterativeImputer

In [86]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy.stats import skew, norm
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
warnings.filterwarnings('ignore')
%matplotlib inline

In [87]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [88]:
train = pd.read_csv('train.csv').drop('Id',axis = 1 )
test = pd.read_csv('test.csv').drop('Id',axis = 1 )
test_Y = pd.read_csv('sample_submission.csv').drop("Id", axis = 1)

In [89]:
train["SalePrice"] = np.log1p(train["SalePrice"])
y_test = np.log1p(test_Y)

In [90]:
train.drop(train[(train['OverallQual']<5) & (train['SalePrice']>200000)].index, inplace=True)
train.drop(train[(train['GrLivArea']>4500) & (train['SalePrice']<300000)].index, inplace=True)
train.reset_index(drop=True, inplace=True)

In [91]:
train_labels = train['SalePrice'].reset_index(drop=True)
train_features = train.drop(['SalePrice'], axis=1)
test_features = test

# Объединяем тестовую и тренировочную выборку
all_features = pd.concat([train_features, test_features]).reset_index(drop=True)
all_features.shape

(2917, 79)

In [92]:
def percent_missing(df):
    data = pd.DataFrame(df)
    df_cols = list(pd.DataFrame(data))
    dict_x = {}
    for i in range(0, len(df_cols)):
        dict_x.update({df_cols[i]: round(data[df_cols[i]].isnull().mean()*100,2)})
    
    return dict_x

missing = percent_missing(all_features)
df_miss = sorted(missing.items(), key=lambda x: x[1], reverse=True)
print('Percent of missing data')
df_miss[0:10]

Percent of missing data


[('PoolQC', 99.69),
 ('MiscFeature', 96.4),
 ('Alley', 93.21),
 ('Fence', 80.43),
 ('FireplaceQu', 48.68),
 ('LotFrontage', 16.66),
 ('GarageYrBlt', 5.45),
 ('GarageFinish', 5.45),
 ('GarageQual', 5.45),
 ('GarageCond', 5.45)]

In [93]:
all_features['MSSubClass'] = all_features['MSSubClass'].apply(str)
all_features['YrSold'] = all_features['YrSold'].astype(str)
all_features['MoSold'] = all_features['MoSold'].astype(str)
all_features['YearRemodAdd'] = all_features['YearRemodAdd'].astype(str)
all_features['YearBuilt'] = all_features['YearBuilt'].astype(str)

In [94]:
def handle_missing(features):
    # the data description states that NA refers to typical ('Typ') values
    features['Functional'] = features['Functional'].fillna('Typ')
    # Replace the missing values in each of the columns below with their mode
    features['Electrical'] = features['Electrical'].fillna("SBrkr")
    features['KitchenQual'] = features['KitchenQual'].fillna("TA")
    features['Exterior1st'] = features['Exterior1st'].fillna(features['Exterior1st'].mode()[0])
    features['Exterior2nd'] = features['Exterior2nd'].fillna(features['Exterior2nd'].mode()[0])
    features['SaleType'] = features['SaleType'].fillna(features['SaleType'].mode()[0])
    features['MSZoning'] = features.groupby('MSSubClass')['MSZoning'].transform(lambda x: x.fillna(x.mode()[0]))
    
    features["PoolQC"] = features["PoolQC"].fillna("None")
    for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
        features[col] = features[col].fillna(0)
    for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
        features[col] = features[col].fillna('None')
    for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
        features[col] = features[col].fillna('None')
        
    features['LotFrontage'] = features.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
    objects = []
    for i in features.columns:
        if features[i].dtype == object:
            objects.append(i)
    features.update(features[objects].fillna('None'))
        
    numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    numeric = []
    for i in features.columns:
        if features[i].dtype in numeric_dtypes:
            numeric.append(i)
    features.update(features[numeric].fillna(0))    
    return features

all_features = handle_missing(all_features)

In [95]:
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric = []
for i in all_features.columns:
    if all_features[i].dtype in numeric_dtypes:
        numeric.append(i)

In [96]:
skew_features = all_features[numeric].apply(lambda x: skew(x)).sort_values(ascending=False)

high_skew = skew_features[skew_features > 0.5]
skew_index = high_skew.index

print("There are {} numerical features with Skew > 0.5 :".format(high_skew.shape[0]))
skewness = pd.DataFrame({'Skew' :high_skew})
skew_features.head(10)

There are 25 numerical features with Skew > 0.5 :


MiscVal          21.939672
PoolArea         17.688664
LotArea          13.109495
LowQualFinSF     12.084539
3SsnPorch        11.372080
KitchenAbvGr      4.300550
BsmtFinSF2        4.144503
EnclosedPorch     4.002344
ScreenPorch       3.945101
BsmtHalfBath      3.929996
dtype: float64

In [97]:
mmx = MinMaxScaler()

In [98]:
# Normalize skewed features
for i in skew_index:
    all_features[i] = boxcox1p(all_features[i], boxcox_normmax(all_features[i] + 1))

In [99]:
for col in all_features.columns[all_features.dtypes != 'O']:
    all_features[col] = mmx.fit_transform(all_features[col].values.reshape(-1,1))

In [100]:
all_features['MSSubClass'] = all_features['MSSubClass'].apply(float)
all_features['YrSold'] = all_features['YrSold'].astype(float)
all_features['MoSold'] = all_features['MoSold'].astype(float)
all_features['YearRemodAdd'] = all_features['YearRemodAdd'].astype(float)
all_features['YearBuilt'] = all_features['YearBuilt'].astype(float)

In [101]:
all_features = pd.get_dummies(all_features).reset_index(drop=True)
all_features.shape

(2917, 302)

In [102]:
all_features = all_features.loc[:,~all_features.columns.duplicated()]

In [103]:
all_features = all_features[['YrSold','MoSold','LotArea','BedroomAbvGr']]

In [104]:
pd.options.display.max_rows = 302
all_features.head().T

,0,1,2,3,4
YrSold,2008.00000,2007.000000,2008.000000,2006.000000,2008.000000
MoSold,2.00000,5.000000,9.000000,2.000000,12.000000
LotArea,0.31522,0.338667,0.368183,0.337702,0.413082
BedroomAbvGr,0.37500,0.375000,0.375000,0.375000,0.500000


In [105]:
all_features.columns[all_features.isna().sum()>0]

Index([], dtype='object')

In [106]:
X_train = all_features.iloc[:len(train_labels), :]
X_test = all_features.iloc[len(train_labels):, :]
X_train.shape, train_labels.shape, X_test.shape

((1458, 4), (1458,), (1459, 4))

In [107]:
LR = LinearRegression()
LR.fit(X_train,train_labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [108]:
y_pred = LR.predict(X_test)

In [110]:
LR.score(X_test,y_test.values)

-0.8721237806201002

In [112]:
metrics.mean_absolute_error(y_test.values,y_pred)

0.08872326349731852

In [113]:
metrics.mean_squared_error(y_test.values,y_pred)

0.015171450943040543

In [114]:
np.sqrt(metrics.mean_squared_error(y_test.values,y_pred))

0.12317244392736772

In [115]:
metrics.r2_score(y_test,y_pred)

-0.8721237806201001

In [120]:
y_test['Predicted'] = pd.Series(y_pred)

In [121]:
y_test

,SalePrice,Predicted
0,12.039298,12.037636
1,12.142917,12.149953
2,12.120431,12.120431
3,12.096919,12.043300
4,11.923253,11.762529
5,12.084763,12.030541
6,12.055666,11.958234
7,12.073181,11.986507
8,11.995430,11.971518
9,11.987464,11.928990
